In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob
import pandas as pd
import re
import string
import numpy as np
from collections import Counter
import statsmodels.api as sm
import calendar

terms = []
file_list = [
   '/content/drive/My Drive/gun/dictionaries/affect_unique.txt',
   '/content/drive/My Drive/gun/dictionaries/moral_unique.txt'
]
for file_name in file_list:
    print(file_name)
    with open(file_name) as fb:
        for line in fb.readlines():
            terms.append(line.replace('*', '').replace('\n', '' ))
 
print(terms)


stopwords = [u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', 
    u"she's", u'her', u'hers', u'herself', u'it', u"it's", u'its', u'itself', u'they', u'them', 
    u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', 
    u'that', u"that'll", u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be',
    u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing',
    u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until',
    u'while', u'of', u'at', 'in', 'to', 'for', 'you', 'no', 'there', 'i', '&amp', "you're", "you've"]
df_tweets = pd.read_csv("/content/drive/My Drive/gun/tweets.csv")
day_count = Counter()
vectors = Counter()
for i, row in df_tweets.iterrows():
    tweet_vector = np.zeros(len(terms))
    words = row['text']
    if not isinstance(words, str):
        continue
    words = words.lower().replace('\n', ' ').split(' ')
    num_words = len(words)
    date = row['date'].split('T')[0]
    for word in words:
        if word[:4] == 'http':
            continue
        if not word:
            continue
        if word[0] in ['#', '@']:
            continue # Diregard Hashtags, mentions and retweets.
        if word == 'rt':
            continue
        if word in stopwords:
            continue
        if word[-3:] == '…':
            continue
        invalidChars = set(string.punctuation.replace("-", ""))
        if any(char in invalidChars for char in word):
            continue
        elif word.isnumeric():
            continue
        elif "…" in word:
            continue
        elif "\"" in word:
            continue
        else:
            if word in terms:
                i = terms.index(word)
                tweet_vector[i] += 1
    date = row['date'].split('T')[0]
    vectors[date] += tweet_vector
    day_count[date] += 1


Y = []
X = []
df_ncis = pd.read_excel('/content/drive/My Drive/gun/NICS_Daily_data_corr.xlsx')
df_ncis = df_ncis[df_ncis['Date'].str.contains("2019")]
for i , row in df_ncis.iterrows():
    day, month, number, year = row['Date'].split()
    month = list(calendar.month_name).index(month)
    month = month if len(str(month)) == 2 else f'0{month}'
    number = number[:-1] if len(number) != 2 else f'0{number[:-1]}'
    date = f'{year}-{month}-{number}'
    for day in vectors:
        if date == day:
            Y.append(row['Daily NICS checks'])
            X.append(vectors[day] / day_count[day])

X = sm.add_constant(X)
est = sm.OLS(Y, X)
est2 = est.fit()
print(est2.summary())

/content/drive/My Drive/gun/dictionaries/affect_unique.txt
/content/drive/My Drive/gun/dictionaries/moral_unique.txt
['abusi', 'accept', 'accepta', 'accepted', 'accepting', 'accepts', 'ache', 'aching', 'active', 'admir', 'ador', 'advantag', 'adventur', 'advers', 'affection', 'afraid', 'aggravat', 'aggress', 'agoniz', 'agony', 'agree', 'agreeab', 'agreed', 'agreeing', 'agreement', 'agrees', 'alarm', 'alone', 'alright', 'amaz', 'amor', 'amus', 'anger', 'angr', 'anguish', 'annoy', 'antagoni', 'anxi', 'aok', 'apath', 'appall', 'appreciat', 'apprehens', 'argh', 'argu', 'arrogan', 'asham', 'assault', 'asshole', 'assur', 'attachment', 'attract', 'aversi', 'avoid', 'award', 'awesome', 'awful', 'awkward', 'bashful', 'bastard', 'battl', 'beaten', 'beaut', 'beloved', 'benefic', 'benevolen', 'benign', 'best', 'better', 'bitch', 'bitter', 'blam', 'bless', 'bold', 'bonus', 'bore', 'boring', 'bother', 'brave', 'bright', 'brillian', 'broke', 'burden', 'calm', 'cared', 'carefree', 'careful', 'careless'

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     6.388
Date:                Wed, 28 Oct 2020   Prob (F-statistic):            0.00408
Time:                        20:18:34   Log-Likelihood:                -1691.2
No. Observations:                 190   AIC:                             3746.
Df Residuals:                       8   BIC:                             4337.
Df Model:                         181                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.407e+04   4109.740     20.455      0.0

In [ ]:
print(est2.rsquared)

0.9931288128627326


In [ ]:
significant = []

for i, p in enumerate(est2.pvalues):
    if i == 0:
        continue # Skip the Intercept
    if p <= 0.05:
        print(p, i, terms[i], est2.params[i])
        significant.append(terms[i])
print(significant)
print(len(terms), len(est2.pvalues), len(est2.params))

0.01164163616946302 1 accept 1.25734444868817e-07
0.038169240403516984 2 accepta 875702.3361415575
0.0010024288773111888 5 accepts 119144.55621133625
0.04179226754235172 7 aching 428589.7756403943
0.005701449972623285 9 admir 936276.824006169
0.01066429342766807 10 ador 1.1751401680743025e-08
0.0389702627299093 11 advantag -4.534063997010599e-09
0.003541090677122425 12 adventur 1.1451357420616012e-08
0.013277293324749024 17 aggress 2.0815986550020584e-08
0.0009828782280539093 21 agreeab -339056.533052665
0.019768830544671476 23 agreeing 477586.9922599334
0.0288571331388533 26 alarm 796434.5505432398
0.0043537570959283544 28 alright -877060.7649585534
0.029052366343134458 30 amor 6.361030167833801e-09
0.024897349767864327 31 amus -7230.080519562129
0.02895380072886691 34 anguish 6.421256016518622e-09
0.012197603325672376 37 anxi 6.197568706434438e-09
0.002716751615809767 43 argh -6.7834096044108944e-09
0.03039434657235322 53 avoid 1.9671869053285804e-09
0.005384575632695728 54 award -13